In [ ]:
# STEP 1: Install and import the program
#
# clone the repo to your local directory
# navigate to the \bifacialvf directory which contains setup
# run pip install -e .

import bifacialvf    
import os
# change directory to \bifacialvf\ root
os.chdir(os.path.dirname(bifacialvf.__file__))
%pwd

In [ ]:
#2. Set the Values of your test
# Remember all values are normalized to your panel length (slope, which will equal 1).
# If your slope is different than 1 m, desired C and D (or Cv and r2r in tracking case) will need to be 
# divided by the slope length.
# i.e.: panel 1mx1.59m, in portrait mode means slope = 1.59. For a height C of 1m, C = 1/1.59. 
#         For a rtr of 1.5m, D=0.51519/1.59 if beta tilt angle = 10 

# Set mandatory variables
beta = 10                   # PV tilt (deg)
sazm = 180                  # PV Azimuth(deg)
C = 1                      # GroundClearance(panel slope lengths)
D = 0.51519                 # DistanceBetweenRows(panel slope lengths) this is NOT row to row spacing
TMYtoread = "data/724010TYA.csv"   # VA Richmond
writefiletitle = "data/Output/test.csv"

# Set optional variables.  These are the default values
rowType = "interior"        # RowType(first interior last single)
transFactor = 0.013         # TransmissionFactor(open area fraction)
cellRows = 6                # CellRows(# hor rows in panel)   This is the number of irradiance values returned along module chord
PVfrontSurface = "glass"    # PVfrontSurface(glass or AR glass)
PVbackSurface = "glass"     # PVbackSurface(glass or AR glass)
albedo = 0.62               # ground albedo beneath system
# 1-axis tracking instructions (optional)
tracking=False
backtrack=True       # backtracking optimization as defined in pvlib
r2r = 1.5                   # row to row spacing in meters. This input is not used (D is used instead) except for in tracking
Cv = 0.05                  # GroundClearance when panel is in vertical position (panel slope lengths)






Notice that fixed systems use the parameter D, which is the space between the end of one panel and the beginning of the end, so it relates to the inclination angle beta. To go from a r2r value to D, you can use

from rtr_and_D_calculation import rtr2D
D = rtr2D(beta, rtr)

In [ ]:
#3. Call the function.



bifacialvf.simulate(TMYtoread, writefiletitle, beta, sazm, 
                C, D, rowType=rowType, transFactor=transFactor, cellRows=cellRows,
                PVfrontSurface=PVfrontSurface, PVbackSurface=PVbackSurface, albedo=albedo, 
                tracking=tracking, backtrack=backtrack, r2r=r2r, Cv=Cv)
    

In [ ]:
#4. Load the results from the resultfile
(data, metadata) = bifacialvf.loadVFresults(writefiletitle)
#print data.keys()
# calculate average front and back global tilted irradiance across the module chord
data['GTIFrontavg'] = data[['No_1_RowFrontGTI', 'No_2_RowFrontGTI','No_3_RowFrontGTI','No_4_RowFrontGTI','No_5_RowFrontGTI','No_6_RowFrontGTI']].mean(axis=1)
data['GTIBackavg'] = data[['No_1_RowBackGTI', 'No_2_RowBackGTI','No_3_RowBackGTI','No_4_RowBackGTI','No_5_RowBackGTI','No_6_RowBackGTI']].mean(axis=1)

# Print the annual bifacial ratio
frontIrrSum = data['GTIFrontavg'].sum()
backIrrSum = data['GTIBackavg'].sum()
print('The bifacial ratio for ground clearance {} and row gap {} is: {:.1f}%'.format(C,D,backIrrSum/frontIrrSum*100))



In [ ]:
# plot the rear irradiance distribution for a single point in time. 1999-07-06
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

data['GTIBackstd'] = data[['No_1_RowBackGTI', 'No_2_RowBackGTI','No_3_RowBackGTI','No_4_RowBackGTI','No_5_RowBackGTI','No_6_RowBackGTI']].std(axis=1)
data['measdatetime'] = pd.to_datetime(data[['Year', 'Month', 'Day','Hour','Minute']])
singleday = (data['measdatetime'] > '1999-07-06') & (data['measdatetime']<'1999-07-07')
singleday2 = (data['measdatetime'] > '1999-07-12') & (data['measdatetime']<'1999-07-13')

fig, ax = plt.subplots()
ax1 = ax.twinx()
ax1.plot(data['measdatetime'][singleday],data['GTIFrontavg'][singleday],'k')
ax1.set_ylabel('Front Irradiance (Wm-2)')
ax.set_ylabel('Rear Irradiance (Wm-2)')
ax.plot(data['measdatetime'][singleday], data['No_1_RowBackGTI'][singleday],'r' , alpha =0.5)
ax.plot(data['measdatetime'][singleday], data['No_2_RowBackGTI'][singleday], 'b', alpha = 0.5)
ax.plot(data['measdatetime'][singleday], data['No_6_RowBackGTI'][singleday], 'g', alpha = 0.5)
ax.set_title('Sunny day')
fig.autofmt_xdate()
fig.tight_layout()


fig2, ax2 = plt.subplots()
ax3 = ax2.twinx()
ax3.plot(data['measdatetime'][singleday2],data['GTIFrontavg'][singleday2],'k')
ax3.set_ylabel('Front Irradiance (Wm-2)')
ax2.set_ylabel('Rear Irradiance (Wm-2)')
ax2.plot(data['measdatetime'][singleday2], data['No_1_RowBackGTI'][singleday2],'r' , alpha =0.5)
ax2.plot(data['measdatetime'][singleday2], data['No_2_RowBackGTI'][singleday2], 'b', alpha = 0.5)
ax2.plot(data['measdatetime'][singleday2], data['No_6_RowBackGTI'][singleday2], 'g', alpha = 0.5)
ax2.set_title('Cloudy day')
fig2.autofmt_xdate()
fig2.tight_layout()




# Example of looping for a Site


In [ ]:

albedoloop=[0.1, 0.2, 0.4, 0.62, 0.8]
Cloop=[0.15, 0.25, 0.5, 0.75, 1]
Rtrloop=[1.5, 1.75, 2, 2.25, 2.5, 3]
betaloop=[5, 10, 15, 20, 25, 30]


for ii in range (0, 4):
    
    if ii==0:
        ctloop=albedoloop
        ctlooptitle='albedo'          
    if ii==1:
        ctloop=Cloop
        ctlooptitle='height'            
    if ii==2:
        ctloop=Rtrloop
        ctlooptitle='rtr'           
    if ii==3:
        ctloop=betaloop
        ctlooptitle='beta'
        
   
    for jj in range (0, len(ctloop)):

        # Base params
        albedo=0.62
        C=0.15
        beta=10
        rtr=1.5
        D = 0.515192246987792

        if ii==0:
            albedo=ctloop[jj]    
        if ii==1:
            C=ctloop[jj]                
        if ii==2:
            D=rtr2D(beta, ctloop[jj])
        if ii==3:
            beta=ctloop[jj]                

    
        writefiletitle="data/Output/"+"TestLoop_"+str(ctlooptitle)+"_"+str(ctloop[jj])+".csv"
       
        
        bifacialvf.simulate(TMYtoread, writefiletitle, beta, sazm, 
                C, D, rowType=rowType, transFactor=transFactor, cellRows=cellRows, 
                albedo=albedo, tracking=tracking, r2r=r2r, Cv=Cv)
